In [ ]:
!pip install dtw-python
!pip install moviepy
!pip install --upgrade moviepy
!pip install imagehash
!pip3 install imageio==2.4.1
!pip install imageio-ffmpeg

import pandas as pd
import numpy as np
import librosa
import librosa.display
from librosa.feature.spectral import mfcc
import matplotlib.pyplot as plt
import warnings
from dtw import dtw
import time
import moviepy.editor as mp
import imagehash
from PIL import Image
import sklearn
import os
warnings.filterwarnings("ignore")

In [ ]:
full_songs_path = '/content/drive/MyDrive/Data mining/fullsongs'

short_songs_path = '/content/drive/MyDrive/Data mining/shorts'

In [ ]:
# find the total duration of the song clip
def findDuration(path):
    song = mp.AudioFileClip(path)
    duration = song.duration
    return duration

In [ ]:

mfcc_df = pd.DataFrame()

mfcc_names = []
for k in range(1, 21):
    mfcc_names.append('MFCC_' + str(k))

count = 0

# total number of songs
total_songs = len(os.listdir(full_songs_path))

for idx in range(total_songs):
    print("Idx is ", idx)
    
    current_full_song = full_songs_path + "/" + sorted(os.listdir(full_songs_path))[idx]
    full_dur = findDuration(current_full_song)
    
    print("current_full_song: ", current_full_song)
    
    current_short_song = short_songs_path + "/" + sorted(os.listdir(short_songs_path))[idx]
    short_dur = findDuration(current_short_song)
    
    mfcc_list = []
    mfcc_mean_list = []
    
    for i in range(int(full_dur - short_dur + 1)):
        # loading song via librosa
        full_amplitude, full_sr = librosa.load(current_full_song, offset = i, duration = short_dur)  

        # calculating MFCC values
        temp_mfcc = librosa.feature.mfcc(full_amplitude, full_sr)
        mfcc_list.append(temp_mfcc)

        # Scaling MFCC values and calculating their means
        mfccs_transformed = sklearn.preprocessing.scale(temp_mfcc, axis=1)
        mfccs_scaled = np.mean(mfccs_transformed.T, axis = 0)
        mfcc_mean_list.append(mfccs_scaled) 
    
    print("Creating dataframe with just mfcc means")
    mfcc_temp_df = pd.DataFrame(mfcc_mean_list, columns = mfcc_names)
        
    # loading short songs via librosa and calculating MFCC values
    short_amplitude, short_sr = librosa.load(current_short_song, offset = 0, duration = short_dur)  
    short_mfcc = librosa.feature.mfcc(short_amplitude, short_sr)
    
    # Dynamic Time Warping 
    dtw_dist_list = []
    print("Finding dtw distance")
    for mfcc in mfcc_list:
        
        dist = dtw(mfcc.T, short_mfcc.T)
        dtw_dist_list.append(dist)
        
    # Calculating Minimum distance
    print("Minimum distance calc")
    min_dist = np.inf
    for counter, i in enumerate(dtw_dist_list):
        
        if(i.normalizedDistance < min_dist):
            min_dist = i.normalizedDistance
            min_idx = counter
    


    for counter, i in enumerate(dtw_dist_list):
        
        if(i.normalizedDistance == min_dist):
            # min_dist = i.normalizedDistance
            print(counter)


    target_vals = []
    for k in range(len(mfcc_mean_list)):
        if k == min_idx:
            target_vals.append(1)
        else:
            target_vals.append(0)
            
    print("Adding target val to the dataframe")
    mfcc_temp_df['target'] = target_vals
    
    print("Appending to the final dataframe")
    mfcc_df = pd.concat([mfcc_df,mfcc_temp_df]).reset_index(drop = True)

In [ ]:
mfcc_df

In [ ]:
mfcc_df.to_csv('/content/drive/MyDrive/Data mining/mfcc_values_0_49.csv', index = False)